# Load pdf book

In [6]:
!pip install PyMuPDF
!pip install textblob
import re
import nltk
# import textblob 
# import PyMuPDF 
import pandas as pd
import matplotlib.pyplot as plt

# Ensure nltk resources are downloaded
nltk.download('punkt')
textblob.download()
PyMuPDF.download()


[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\avrsa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


NameError: name 'textblob' is not defined